## Testing prompt

In [1]:
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted

/home/dinhln/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
genai.configure(api_key="*")
model = genai.GenerativeModel('gemini-1.5-pro')

In [15]:
response = model.generate_content("abc", 
                                    generation_config=genai.types.GenerationConfig(temperature=0.6),
            )
response.text

In [16]:
import time 
from loguru import logger
#Wrapper function
def get_quote(quotes, suggest_topics):

    safe = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]
    
    topic = ["Life and Inspiration: (Includes broader themes of life lessons, inspiration, courage, and personal growth.)",
          "Love and Relationships (Covers themes like love, romance, heartbreak, marriage, and relationships.)",
          "Friendship: (Covers themes like friend, companionship, loyalty, trust, mutual support, and shared experiences.))",
          "Spirituality and Faith: (Includes themes of faith, hope, spiritual growth, and introspection.)",
          "Humor and Light-heartedness: (Covers themes of humor, wit, light-hearted moments, and laughter.)",
          ]

                                        
    prompt_template = f"""
    You are a language model capable of translating quotes from English to Vietnamese that are meaningful and inspirational.

    
    Perform the following actions:
    Step 1 - 1 - Proofread and correct the original quote, paying special attention to punctuation, and rewrite the corrected version of the original quote. Ensure all necessary punctuation marks are correctly placed. If you don't find any errors, just go ahead to step 2.
    Step 2 - Based on the suggested topic below, determine whether each item in the following list of topics is a topic in the original quote. List of topics: {", ".join(topic)}
    Step 3 - Translate the original quote into Vietnamese.
    Step 4 - Proofread and rewrite the corrected version of the translated Vietnamese quote, including punctuation. If you don't find any errors, just go ahead.

    Finally, return the result as a JSON object in the exact format shown below (on a single line without line breaks):
    {{ "topic": "Life and Inspiration", "quote": "Làm việc chăm chỉ là chìa khóa để thành công." }}

    Important notes:
    - If the quote is related to topics like love, inspiration, life, or motivation, translate it in a thoughtful and meaningful manner.
    - If the meaning of the quote is humorous, translate it in a way that maintains its humor and makes people laugh.
    - Do not add any extra characters, explanations, formatting, or line breaks. The result should be returned in the exact same format as the example above on a single line.
    - The suggested topic is not always true, so you should keep this as a reference and prioritize determining the topic based on the meaning of the quote itself.
    - If the determined topic is not in the list of topics, return the topic as None.
    -- **Ensure that all punctuation marks in the quote are correct and appropriately placed. Correct any punctuation errors before proceeding.**

    For each of the following 5 quotes, perform the steps described above and return a list of dictionaries:

    {[{'original_quote': x, 'suggested_topic': y} for x, y in zip(quotes, suggest_topics)]}
    
    """
    start_time = time.time()

    while True:
        
        try:
            print(prompt_template)
            response = model.generate_content(prompt_template, 
                                    generation_config=genai.types.GenerationConfig(temperature=0.6),
                                    safety_settings=safe
            )
            return response.text
        except ResourceExhausted:
            logger.error("Resource exhausted. Retrying in 5s...")
            time.sleep(5)
        
        except Exception as e:
            logger.error(f"Error: {e}")
            time.sleep(5)
        
        current_time = time.time()
        elapsed_time = current_time - start_time
        if elapsed_time >= 300:
            logger.critical("Timeout")
            return None
            break  # Exit the loop once the duration is reached

## Transforming data

In [4]:
from datasets import load_dataset

quote_dataset = load_dataset("csv", data_files="quotes.csv", delimiter=",")

In [5]:
quote_sample = quote_dataset["train"].shuffle(seed=42).select(range(10000))

In [6]:
del quote_dataset

In [18]:
import time 
import json
import re
from loguru import logger
i = logger.add("logs/quote_translated.log")
batch_size = 5
sample_size = 2000
results = []
logger.info("Start translating quotes with batch size is 5...".upper())
for i in range(390,2001):


    logger.info(f"Start batch {i}...")

    batch_quote = quote_sample[i*batch_size:(i+1)*batch_size]

    response = get_quote(batch_quote["quote"], batch_quote["category"])

    if response:

        try:
            response = re.findall(r'\[.*\]', response)[0]

            json_response = json.loads(response)

        except:
            logger.error(f"Error in parsing response. Retrying...")

            logger.error(f"Error response (retrying): {response}")

            response = get_quote(batch_quote["quote"], batch_quote["category"])

            response = re.findall(r'\[.*\]', response)[0]

            try:
                json_response = json.loads(response)

            except:

                logger.critical("Error response (skipping): {response}")

                logger.critical(f"Error in parsing response. Skip batch {i}.")

                continue

        results.extend(json_response)

        if i!=0 and i%10 == 0:

            logger.info(f"Save step at batch {i}...")

            with open(f"data/translated_quote_batch_{i}_{i+10}.json", "w") as f:
                json.dump(results, f, indent=2)

            results= []

            logger.success(f"Done saving translated batch {i}_{i+10}.")

        logger.success(f"Batch {i} done.")

    else:
        logger.critical(f"Can not process anymore. Stop immediately at batch {i}...")
        logger.remove(i)
        continue
    

2024-07-09 12:44:39.499 | INFO     | __main__:<module>:9 - START TRANSLATING QUOTES WITH BATCH SIZE IS 5...
2024-07-09 12:44:39.501 | INFO     | __main__:<module>:13 - Start batch 0...
2024-07-09 12:44:50.262 | SUCCESS  | __main__:<module>:59 - Batch 0 done.
2024-07-09 12:44:50.265 | INFO     | __main__:<module>:13 - Start batch 1...
2024-07-09 12:44:56.565 | SUCCESS  | __main__:<module>:59 - Batch 1 done.
2024-07-09 12:44:56.568 | INFO     | __main__:<module>:13 - Start batch 2...
2024-07-09 12:45:05.422 | SUCCESS  | __main__:<module>:59 - Batch 2 done.
2024-07-09 12:45:05.425 | INFO     | __main__:<module>:13 - Start batch 3...
2024-07-09 12:45:18.889 | SUCCESS  | __main__:<module>:59 - Batch 3 done.
2024-07-09 12:45:18.891 | INFO     | __main__:<module>:13 - Start batch 4...
2024-07-09 12:45:19.298 | ERROR    | __main__:get_quote:70 - Resource exhausted. Retrying in 5s...
2024-07-09 12:45:24.734 | ERROR    | __main__:get_quote:70 - Resource exhausted. Retrying in 5s...
2024-07-09 12

## Multi-Threading

In [9]:
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted
import time 
from loguru import logger
import re
import json
import concurrent.futures

def get_quote(model, quotes, suggest_topics, i ):

    safe = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]
    
    topic = ["Life and Inspiration: (Includes broader themes of life lessons, inspiration, courage, and personal growth.)",
          "Love and Relationships (Covers themes like love, romance, heartbreak, marriage, and relationships.)",
          "Friendship: (Covers themes like friend, companionship, loyalty, trust, mutual support, and shared experiences.))",
          "Spirituality and Faith: (Includes themes of faith, hope, spiritual growth, and introspection.)",
          "Humor and Light-heartedness: (Covers themes of humor, wit, light-hearted moments, and laughter.)",
          ]

                                        
    prompt_template = f"""
    You are a language model capable of translating quotes from English to Vietnamese that are meaningful and inspirational.

    
    Perform the following actions:
    Step 1 - 1 - Proofread and correct the original quote, paying special attention to punctuation, and rewrite the corrected version of the original quote. Ensure all necessary punctuation marks are correctly placed. If you don't find any errors, just go ahead to step 2.
    Step 2 - Based on the suggested topic below, determine whether each item in the following list of topics is a topic in the original quote. List of topics: {", ".join(topic)}
    Step 3 - Translate the original quote into Vietnamese.
    Step 4 - Proofread and rewrite the corrected version of the translated Vietnamese quote, including punctuation. If you don't find any errors, just go ahead.

    Finally, return the result as a JSON object in the exact format shown below (on a single line without line breaks):
    {{ "topic": "Life and Inspiration", "quote": "Làm việc chăm chỉ là chìa khóa để thành công." }}

    Important notes:
    - If the quote is related to topics like love, inspiration, life, or motivation, translate it in a thoughtful and meaningful manner.
    - If the meaning of the quote is humorous, translate it in a way that maintains its humor and makes people laugh.
    - Do not add any extra characters, explanations, formatting, or line breaks. The result should be returned in the exact same format as the example above on a single line.
    - The suggested topic is not always true, so you should keep this as a reference and prioritize determining the topic based on the meaning of the quote itself.
    - If the determined topic is not in the list of topics, return the topic as None.
    -- **Ensure that all punctuation marks in the quote are correct and appropriately placed. Correct any punctuation errors before proceeding.**

    For each of the following 5 quotes, perform the steps described above and return a list of dictionaries:

    {[{'original_quote': x, 'suggested_topic': y} for x, y in zip(quotes, suggest_topics)]}
    
    """
    start_time = time.time()

    while True:
        
        try:
            response = model.generate_content(prompt_template, 
                                    generation_config=genai.types.GenerationConfig(temperature=0.6),
                                    safety_settings=safe
            )
            return response.text
        except ResourceExhausted:
            logger.error(f"Batch {i}: Resource exhausted. Retrying in 5s...")
            time.sleep(5)
        
        except Exception as e:
            logger.error(f"Error: {e}")
            return None
        
        current_time = time.time()
        elapsed_time = current_time - start_time
        if elapsed_time >= 300:
            logger.critical(f"Batch {i}: Timeout")
            return None

def process_batch(model, batch_quote, i):

    logger.info(f"Start batch {i}...")

    response = get_quote(model, batch_quote["quote"], batch_quote["category"], i)

    if response:

        try:
            response = re.findall(r'\[.*\]', response)[0]

            json_response = json.loads(response)

            logger.success(f"Batch {i} done.")

            return json_response

        except:
            logger.error(f"Batch {i}: Error in parsing response. Retrying...")

            logger.error(f"Batch {i}: Error response (retrying): {response}")

            response = get_quote(model, batch_quote["quote"], batch_quote["category"])

            response = re.findall(r'\[.*\]', response)[0]

            try:
                json_response = json.loads(response)

                logger.success(f"Batch {i} done.")

                return json_response

            except:

                logger.critical("Batch {i}: Error response (skipping): {response}")

                logger.critical(f"Batch {i}: Error in parsing response. Skip batch {i}.")
                json_response = []

                return json_response


    else:
        logger.critical(f"Can not process anymore. Skip batch {i}...")
        return []


def translate_and_save_data(model, start_point, end_ponint, quote_sample, num_thread):
    k =logger.add("../logs/quote_translated.log")
    batch_size = 5
    results = []
    logger.info("Start translating quotes with batch size is 5 in multi-threading mode...".upper())

    for i in range(start_point, end_ponint, num_thread):

        logger.info(f"Start batch {i}-->{i+num_thread} in multi-threading...")

        with concurrent.futures.ThreadPoolExecutor() as executor:
            responses = executor.map(process_batch, [model] *num_thread,[quote_sample[(i+j)*batch_size:((i+j)+1)*batch_size] for j in range(num_thread)], [i+j for j in range(num_thread)])
        for response in responses:
            results.extend(response)

        logger.info(f"Save step at batch {i+j}...")
        with open(f"../data/translated_quote_batch_{i}_{i+num_thread}.json", "w") as f:
            json.dump(results, f, indent=2)
        results= []
        logger.success(f"Done saving translated batch {i}_{i+num_thread}.")
    logger.success(f"Batch {i}-->{i+num_thread} done.")


translate_and_save_data(model, 0, 10, quote_sample)


2024-07-10 17:17:56.341 | INFO     | __main__:translate_and_save_data:141 - START TRANSLATING QUOTES WITH BATCH SIZE IS 5 IN MULTI-THREADING MODE...
2024-07-10 17:17:56.342 | INFO     | __main__:translate_and_save_data:145 - Start batch 0-->10 in multi-threading...
2024-07-10 17:17:56.345 | INFO     | __main__:process_batch:91 - Start batch 0...
2024-07-10 17:17:56.347 | INFO     | __main__:process_batch:91 - Start batch 1...
2024-07-10 17:17:56.348 | INFO     | __main__:process_batch:91 - Start batch 2...
2024-07-10 17:17:56.349 | INFO     | __main__:process_batch:91 - Start batch 3...
2024-07-10 17:17:56.350 | INFO     | __main__:process_batch:91 - Start batch 4...


2024-07-10 17:17:56.350 | INFO     | __main__:process_batch:91 - Start batch 5...
2024-07-10 17:17:56.351 | INFO     | __main__:process_batch:91 - Start batch 6...
2024-07-10 17:17:56.352 | INFO     | __main__:process_batch:91 - Start batch 7...
2024-07-10 17:17:56.352 | INFO     | __main__:process_batch:91 - Start batch 8...
2024-07-10 17:17:56.353 | INFO     | __main__:process_batch:91 - Start batch 9...
2024-07-10 17:17:56.696 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s...
2024-07-10 17:17:56.697 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s...
2024-07-10 17:17:56.699 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s...
2024-07-10 17:17:56.699 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s...
2024-07-10 17:17:56.769 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s...
2024-07-10 17:18:01.935 | ERROR    | __main__:get_quote:75 - Resource exhausted. Retrying in 5s

[{'topic': 'Spirituality and Faith', 'quote': 'Con người có thể lừa dối đồng loại, sự dối trá có thể nối tiếp sự dối trá, và con cái của kẻ ác có thể có sức mạnh để dụ dỗ những kẻ ngu ngốc và chưa được dạy dỗ, cho đến khi không còn gì ngoài hư cấu nuôi sống nhiều người, và quả của sự giả dối mang theo dòng chảy của nó những kẻ bồng bột đến nấm mồ; nhưng một cái chạm của ngón tay yêu thương của Ngài, vâng, một tia vinh quang từ thế giới thượng giới, hoặc một lời từ miệng của Đấng Cứu Rỗi, từ lòng của cõi vĩnh hằng, sẽ đánh bật tất cả vào sự vô nghĩa, và xóa sạch nó mãi mãi khỏi tâm trí.'}, {'topic': 'Life and Inspiration', 'quote': 'Cậu đã bao giờ đi tàu lượn siêu tốc chưa, Togawa-kun? Cậu chỉ ngồi trên đó vài phút thôi, phải không? Nếu cậu dành toàn bộ thời gian đi xe để nghĩ "chỉ còn vài giây nữa... cho đến khi chuyến đi này kết thúc..." vậy thì việc đi xe ngay từ đầu để làm gì? Không ai biết tại sao chúng ta sống. Chúng ta không có thời gian để bận tâm.'}, {'topic': 'Life and Inspira

## EDA

In [1]:
from datasets import load_dataset

quote_dataset = load_dataset("json", data_files="../merged_data/translated_quote.json")



/home/dinhln/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 167587 examples [00:01, 88389.95 examples/s]


In [2]:
# Remove None and other noisy topic
quote_dataset = quote_dataset.filter(lambda x: x["topic"] in ["Life and Inspiration", "Love and Relationships", "Friendship", "Humor and Light-heartedness"])
len(quote_dataset["train"])

Filter: 100%|██████████| 167587/167587 [00:01<00:00, 117243.86 examples/s]


113637

In [3]:
# Only keep the quote and topic columns
quote_dataset = quote_dataset.select_columns(["quote", "topic"])

In [4]:
# Unescape HTML characters
import html

quote_dataset = quote_dataset.map(lambda x: {"quote": html.unescape(x["quote"])})
len(quote_dataset["train"])

Map: 100%|██████████| 113637/113637 [00:05<00:00, 19716.63 examples/s]


113637

In [5]:
# Remove toxic word
quote_dataset = quote_dataset.filter(lambda x: ("thằng khốn" or "khốn khiếp") not in x["quote"])
len(quote_dataset["train"])

Filter: 100%|██████████| 113637/113637 [00:00<00:00, 207520.42 examples/s]


113627

In [6]:
# Remove duplicate quotes
print(len(quote_dataset.unique("quote")["train"]))

def get_hash(example):
    """Get hash of content field."""
    return {"hash": hash(example["quote"])} # can use any hashing function here
    
def check_uniques(example, uniques):
    """Check if current hash is still in set of unique hashes and remove if true."""
    if example["hash"] in uniques:
        uniques.remove(example["hash"])
        return True
    else:
        return False

quote_dataset = quote_dataset.map(get_hash)

uniques = set(quote_dataset["train"].unique("hash"))

quote_dataset = quote_dataset.filter(check_uniques, fn_kwargs={"uniques": uniques})
del uniques

Flattening the indices: 100%|██████████| 113627/113627 [00:01<00:00, 69120.25 examples/s]


112875


Filter: 100%|██████████| 113627/113627 [00:00<00:00, 205230.64 examples/s]


In [9]:
quote_dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'topic', 'hash'],
        num_rows: 112875
    })
})

In [10]:
def rename_topic_to_vietmanese(topic):
    if topic == "Life and Inspiration":
        return "Cuộc sống và nguồn cảm hứng"
    elif topic == "Love and Relationships":
        return "Tình yêu và các mối quan hệ"
    elif topic == "Friendship":
        return "Tình bạn"
    elif topic == "Humor and Light-heartedness":
        return "Hài hước và vui vẻ"
    else:
        return topic

quote_dataset = quote_dataset.map(lambda x: {"topic": rename_topic_to_vietmanese(x["topic"])})

Map: 100%|██████████| 112875/112875 [00:06<00:00, 18417.83 examples/s]


In [17]:
quote_dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'topic', 'hash'],
        num_rows: 112875
    })
})

In [16]:
# Push to hub
import os
quote_dataset.push_to_hub("dinhlnd1610/Vietnamese_Quote_Dataset_100K", token = "*")

Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.35s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/dinhlnd1610/Vietnamese_Quote_Dataset_100K/commit/7a1adab87e9c8ee93ba4b093ffa8eb7c4ec0ff03', commit_message='Upload dataset', commit_description='', oid='7a1adab87e9c8ee93ba4b093ffa8eb7c4ec0ff03', pr_url=None, pr_revision=None, pr_num=None)

## Training

In [19]:
# Test data collator function
from trl import DataCollatorForCompletionOnlyLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../gemma-2b", use_fast=True,padding_side='right')

collator = DataCollatorForCompletionOnlyLM(
        instruction_template="<bos>",
        response_template="<sep>",
        tokenizer=tokenizer,
        mlm=False,
    )



[2024-07-15 15:29:10,987] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [21]:
text = {"quote": "Tôi và bạn","topic" : "Cuộc sống và những người bạn"}
ids = tokenizer.apply_chat_template(text,tokenize=True)

In [22]:
collator([{"input_ids": ids}])

{'input_ids': tensor([[     2,  10493,  22247,  42064,   5581,  15319,  12301,  15026, 256000,
         117709,   5581,  15026,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
         117709,   5581,  15026,      1]])}

In [23]:
# Check 
tokenizer.decode([ 117709,   5581,  15026])

'Tôi và bạn'

: 

In [60]:
def convert_conversation_to_input(examples):
        input_ids = tokenizer.apply_chat_template(examples, tokenize=True)[: 8096]
        return {'input_ids': input_ids}

quote_sample = quote_dataset["train"].shuffle(seed=42).select(range(15))

In [67]:
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{{'<bos>' + messages['topic'] + '<sep>' + messages['quote'] + '<eos>'}}{% if add_generation_prompt %}{{'<sep>'}}{% endif %}"

## Test model

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = '/root/Project/unsloth-lora-checkpoints/checkpoint-82500'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16, 
    device_map="cuda",
    use_cache=True,
)

eos_token_id = tokenizer.convert_tokens_to_ids(["<eos>"])
print("eos_token_id: ", eos_token_id)
human = "Cuộc sống"
conversation = ["<bos>Truyền cảm hứng học tập<sep>Cha mẹ"]
input_ids = tokenizer(conversation, return_tensors="pt").to(model.device)

out_ids = model.generate(
    input_ids=input_ids["input_ids"],
    max_new_tokens=256,
    do_sample=True,
    top_p=0.9,
    top_k=40,
    temperature=0.7,
    eos_token_id=eos_token_id,
)
assistant = tokenizer.batch_decode(out_ids[:, input_ids["input_ids"].size(1): ], skip_special_tokens=True)[0].strip()
print(assistant)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

eos_token_id:  [1]
, hãy nói với con cái của bạn rằng: "Chúng ta có thể làm những điều kỳ diệu.


In [ ]:
from peft import PeftModel
base_model_path = "../gemma-2b"
lora_path = "../unsloth-lora-checkpoints/checkpoint-82500"
base = AutoModelForCausalLM.from_pretrained(
    base_model_path, torch_dtype=torch.bfloat16,
)
lora_model = PeftModel.from_pretrained(base, lora_path)
model = lora_model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(base_model_path)



: 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../gemma-2b")

/home/dinhln/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer.push_to_hub("dinhlnd1610/ahihi", token = '*')

tokenizer.json: 100%|██████████| 17.5M/17.5M [00:04<00:00, 3.95MB/s]


CommitInfo(commit_url='https://huggingface.co/dinhlnd1610/ahihi/commit/a8dc461447fee51b339ea3c99cb8933613fb5c4d', commit_message='Upload tokenizer', commit_description='', oid='a8dc461447fee51b339ea3c99cb8933613fb5c4d', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
from openai import OpenAI
import os

# Initialize the OpenAI Client with your RunPod API Key and Endpoint URL
client = OpenAI(
    api_key="*",
    base_url=f"https://api.runpod.ai/v2/vllm-6pvh6kmjnix3md/openai/v1",
)

In [18]:

chat_completion = client.chat.completions.create(
    model="dinhlnd1610/ahihi",
    messages={"quote": "","topic" : "Cuộc sống"}
)

In [19]:
chat_completion

ChatCompletion(id=None, choices=None, created=None, model=None, object='error', system_fingerprint=None, usage=None, code=400, message="21 validation errors for ChatCompletionRequest\nmessages.0.typed-dict.content\n  Field required [type=missing, input_value={'quote': '', 'topic': 'Cuộc sống'}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.7/v/missing\nmessages.0.typed-dict.role\n  Field required [type=missing, input_value={'quote': '', 'topic': 'Cuộc sống'}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.7/v/missing\nmessages.0.typed-dict.quote\n  Extra inputs are not permitted [type=extra_forbidden, input_value='', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.7/v/extra_forbidden\nmessages.0.typed-dict.topic\n  Extra inputs are not permitted [type=extra_forbidden, input_value='Cuộc sống', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.7/v/extra_f

In [6]:
from openai import OpenAI
import os
import time

endpoint_id = "vllm-w9uf0ka28xb83b"
api_key = "*"

client = OpenAI(
    base_url=f"https://api.runpod.ai/v2/{endpoint_id}/openai/v1",
    api_key=api_key,
)
stream = False

generation_time = time.time()

completion = client.completions.create(
    model="dinhlnd1610/ahihi",
    prompt=["<bos>Cuộc sống và truyền cảm hứng<sep>Đôi khi"],
    max_tokens=256,
    temperature=0.7,
    top_p=0.9,
    n=1,  # Number of completions to generate
    stream=stream,
)

generation_duration = time.time() - generation_time

if stream:
    for c in completion:
        print(c.choices[0].text, end = "", flush = True)
else:
    print(completion.choices[0].text)
print(f"Generation duration: {generation_duration:.2f}s")

, bạn sẽ không bao giờ tìm thấy một khoảnh khắc trong cuộc sống mà bạn không thể trở lại.
Generation duration: 41.90s
